**ÉTAPE 0** : préparation des données

In [1]:
import process 
import pandas as pd
import numpy as np

# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv'
# Jerome path : 'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DATA_Statapp.csv'
# Mohamed path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv'
df = pd.read_csv('/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv')

# Apply conversion function to 'open' and 'close' columns
df['open'] = df['open'].apply(process.safe_literal_eval)
df['close'] = df['close'].apply(process.safe_literal_eval)

# Calculate returns for each line
df['return'] = df.apply(lambda row: [(close - open) / open for open, close in zip(row['open'], row['close'])], axis=1)

new_df = df[['ticker', 'return']] # create a new data frame with the column ticker and return 

# Créons le DataFrame à partir des listes dans 'return'
# On suppose ici que 'new_df' est déjà défini et contient la colonne 'return'

# Convertir chaque liste dans la colonne 'return' en plusieurs colonnes dans le nouveau DataFrame
returns_df = pd.DataFrame(new_df['return'].tolist())

# Ajouter la colonne 'ticker' du 'new_df' au début de 'returns_df'
returns_df.insert(0, 'ticker', new_df['ticker'])

# Renommer les colonnes pour refléter qu'elles sont des rendements
returns_df.columns = ['ticker'] + [f'return_{i}' for i in range(len(returns_df.columns) - 1)]

df_cleaned = process.remove_rows_with_nan(returns_df)
df_cleaned.reset_index(drop=True, inplace=True)

process.check_nan_inf(df_cleaned)

df_cleaned.set_index('ticker', inplace=True) ## ces deux lignes sont fondamentales, ne pas les retirer !!
df.set_index('ticker', inplace=True)


There are no NaN values in the dataframe


In [9]:
import process 
lookback_window = [5000, 5250]
corr_matrix = process.correlation_matrix(lookback_window=lookback_window, df_cleaned=df_cleaned)
cluster_composition_and_centroid = process.cluster_composition_and_centroid(df_cleaned=df_cleaned, correlation_matrix=corr_matrix, number_of_clusters=20, lookback_window=lookback_window)
constituent_weights = process.constituent_weights(
    df_cleaned=df_cleaned, 
    cluster_composition=cluster_composition_and_centroid, 
    sigma=10, 
    lookback_window=lookback_window
)
cluster_return = process.cluster_return(constituent_weights=constituent_weights, df_cleaned=df_cleaned, df=df, lookback_window=lookback_window)

In [10]:
cov_matrix = cluster_return.transpose().cov()
cluster_return_target = process.cluster_return(constituent_weights=constituent_weights, df_cleaned=df_cleaned, df=df, lookback_window=[lookback_window[1], lookback_window[1]+1])
expected_returns = process.noised_array(y=cluster_return_target, eta=0.01).iloc[:, 0].values.squeeze()

In [13]:
expected_returns.mean()

0.538109308693756

In [16]:
from pypfopt.efficient_frontier import EfficientFrontier

ef = EfficientFrontier(expected_returns=expected_returns, cov_matrix=cov_matrix, weight_bounds=(0, 1))
ef. efficient_return(target_return=expected_returns.mean())
ef.clean_weights()

OrderedDict([('cluster 1', 0.0),
             ('cluster 2', 0.0),
             ('cluster 3', 0.20814),
             ('cluster 4', 0.16552),
             ('cluster 5', 0.0),
             ('cluster 6', 0.0),
             ('cluster 7', 0.05284),
             ('cluster 8', 0.0),
             ('cluster 9', 0.23842),
             ('cluster 10', 0.0),
             ('cluster 11', 0.07347),
             ('cluster 12', 0.0),
             ('cluster 13', 0.0),
             ('cluster 14', 0.0),
             ('cluster 15', 0.0),
             ('cluster 16', 0.10912),
             ('cluster 17', 0.0),
             ('cluster 18', 0.0),
             ('cluster 19', 0.1525)])

In [ ]:
markowitz_weights = process.markowitz_weights(cluster_return_res=cluster_return, constituent_weights=constituent_weights, df_cleaned=df_cleaned, df=df, lookback_window=lookback_window, eta=0.01)
final_weights = process.final_weights(markowitz_weights=markowitz_weights, constituent_weights=constituent_weights)
consolidated_W = process.consolidated_W(number_of_repetitions=2, lookback_window=lookback_window, df_cleaned=df_cleaned, number_of_clusters=20, sigma=10e-2, df=df, eta=0.01)

In [7]:
portfolio_return = process.portfolio_returns(evaluation_window=10, df_cleaned=df_cleaned, lookback_window=lookback_window, consolidated_W=consolidated_W, df=df)

In [8]:
import process
import warnings 

warnings.filterwarnings('ignore')

lookback_window = [5000, 5250]
number_of_clusters = 20
sigma = 10e-3
clustering_method = 'SPONGE' 
number_of_repetition = 20
evaluation_window = 10
eta=0.01

PnL = []
overall_return = pd.DataFrame()
portfolio_value=[1]

consolidated_W = process.consolidated_W(number_of_repetitions=10, lookback_window=lookback_window, df_cleaned=df_cleaned, number_of_clusters=number_of_clusters, sigma=sigma, df=df, eta=eta)

portfolio_return = process.portfolio_returns(evaluation_window=evaluation_window, df_cleaned=df_cleaned, lookback_window=lookback_window, consolidated_W=consolidated_W, df=df)

overall_return = pd.concat([overall_return, portfolio_return])

PnL = np.concatenate((PnL, np.reshape(np.cumprod(1 + portfolio_return)*portfolio_value[-1] - portfolio_value[-1], (evaluation_window,))))## car on réinvestit immédiatement après 
portfolio_value.append(portfolio_value[-1]+PnL[-1])

ValueError: Quadratic form matrices must be symmetric/Hermitian.